## Panorama Stitching using SuperGlue
SuperGlue network is a Graph Neural Network combined with an Optimal Matching layer that is trained to perform matching on two sets of sparse image features.

***Source Images from Open Adobe Project***
[Adobe Panoramas](https://liquidtelecom.dl.sourceforge.net/project/adobedatasets.adobe/adobe_panoramas.tgz)

In [37]:
import numpy as np
import cv2 as cv
import matplotlib.pyplot as plt
%config IPCompleter.greedy=True
%config Completer.use_jedi = False

In [27]:
# generating the necessary txt file to input for the super glue algorithm
img_name = 'top_view' # set of outdoor images
num_images = 1200
# Order of the images. To stitch left and right images as depicted in the below
order = range(num_images -1,0,-1) 
with open('adobe_panorama.txt', 'w') as file:
    for i in order:
        file.write("{img}-{:01}.png {img}-{:01}.png\n".format(i,i-1, img = img_name))
        


In [28]:
# Running the super glue algorithm on outdoor dataset to match features
"""
    Do not resize the images, 
    If you need to use the extracted keypoints,
    to process the "original" source images.

    Set the parameter to -1 to avoid resizing
    --resize -1
    
    Other parameter can be set as folows,
    to extract the best matching features.
    * Non maximum Suppression Radius = 5
    * Matching Confidence = 0.9
"""
!python3 match_pairs.py  --resize -1\
                        --superglue outdoor \
                        --max_keypoints 2048 \
                        --nms_radius 5 \
                        --resize_float \
                        --input_dir adobe_panorama/ \
                        --input_pairs adobe_panorama.txt \
                        --output_dir adobe_panorama/output \
                        --viz \
                        --keypoint_threshold 0.05 \
                        --match_threshold 0.9

Namespace(input_pairs='adobe_panorama.txt', input_dir='adobe_panorama/', output_dir='adobe_panorama/output', max_length=-1, resize=[-1], resize_float=True, superglue='outdoor', max_keypoints=2048, keypoint_threshold=0.05, nms_radius=5, sinkhorn_iterations=20, match_threshold=0.9, viz=True, eval=False, fast_viz=False, cache=False, show_keypoints=False, viz_extension='png', opencv_display=False, shuffle=False, force_cpu=False)
Will not resize images
Running inference on device "cpu"
Loaded SuperPoint model
Loaded SuperGlue model ("outdoor" weights)
Looking for data in directory "adobe_panorama"
Will write matches to directory "adobe_panorama/output"
Will write visualization images to directory "adobe_panorama/output"
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/torch/nn/functional.py:4296: UserWarning: Default grid_sample and affine_grid behavior has changed to align_corners=False since 1.3.0. Please specify align_corners=True if the old behavior is des

In [29]:
#generatig the npz files for extract matching information
npz_files = ["{img}-{:01}_{img}-{:01}_matches.npz".format(i,i-1, img = img_name) for i in order]
for file in npz_files:
    path = 'adobe_panorama/output/'+file
    npz = np.load(path)
print(npz.files)

['keypoints0', 'keypoints1', 'matches', 'match_confidence']


For each keypoint in `keypoints0`, the `matches` array indicates the index of the matching keypoint in `keypoints1`, or `-1` if the keypoint is unmatched.

In [42]:
# extracting information from the npz files 
def loadNPZ(npz_file):    
    npz = np.load('adobe_panorama/output/'+ npz_file)
    point_set1 = npz['keypoints0'][npz['matches']>-1]
    matching_indexes =  npz['matches'][npz['matches']>-1] # -1 if the keypoint is unmatched
    point_set2 = npz['keypoints1'][matching_indexes]
    print("Number of matching points for the findHomography algorithm:")
    print("In left  image:", len(point_set1),"\nIn right image:", len(point_set2))
    return point_set1, point_set2

In [31]:
def pltSourceImages(imageSet):    
    im_left = cv.imread('adobe_panorama/top_view-{:01}.png'.format(imageSet),cv.IMREAD_ANYCOLOR)
    im_right = cv.imread('adobe_panorama/top_view-{:01}.png'.format(imageSet -1),cv.IMREAD_ANYCOLOR)
    
    # Marking the detected features on the two images.
    for point in point_set1.astype(np.int32):
        cv.circle(im_left, tuple(point), radius=8, color=(255, 255, 0), thickness=-1)

    for point in point_set2.astype(np.int32):
        cv.circle(im_right, tuple(point), radius=8, color=(255, 255, 0), thickness=-1)

    fig = plt.figure(figsize = (10,10))
    plt.subplot(121),plt.imshow(im_left, cmap='gray', vmin = 0, vmax = 255)
    plt.subplot(122),plt.imshow(im_right, cmap='gray', vmin = 0, vmax = 255)
    plt.show()

In [32]:
def plotMatches(imageSet):
    plt.figure(figsize=(10,10))
    matched_points = cv.imread('adobe_panorama/output/top_view-{:01}_top_view-{:01}_matches.png'.\
                     format(imageSet, imageSet -1),cv.IMREAD_ANYCOLOR)
    plt.imshow(matched_points, cmap='gray', vmin = 0, vmax = 255)
    plt.show()

In [ ]:
panoramas = list()

for imgSet in range(num_images-1,1,-1):  
    # loading points
    point_set1, point_set2 = loadNPZ(npz_files[num_images-1 -imgSet])
    # pltSourceImages(imgSet)
    # plotMatches(imgSet)    
    # getting the required source images
    im_left = cv.imread('adobe_panorama/top_view-{:01}.png'.format(imgSet),cv.IMREAD_ANYCOLOR)
    im_right = cv.imread('adobe_panorama/top_view-{:01}.png'.format(imgSet -1),cv.IMREAD_ANYCOLOR)
    #find Homography between two source images
    H, status = cv.findHomography(point_set1, point_set2, cv.RANSAC, 5.0) 
    # Prints the Homography matrix that transform left image to right image
    # print(H) 
    # Applies a homogeneous transformation to an image.
    # To transform the right image to left we need to consider the inverse.
    panorama = cv.warpPerspective(im_right, np.linalg.inv(H), (1500,800)) 
    # plt.figure(figsize=(10,10))
    # plt.imshow(panorama, cmap='gray', vmin = 0, vmax = 255)
    panorama[0:im_left.shape[0], 0:im_left.shape[1]] = im_left
    panoramas.append(panorama)
    # plt.figure(figsize=(10,10))
    # plt.imshow(panorama, cmap='gray', vmin = 0, vmax = 255)
    # plt.show()    
    # print("-"*100)
cv.imwrite('panorama.jpg', panorama)

In [45]:
# create a video from the generated panorama in mp4 format
height, width, layers = panorama.shape
size = (width,height)
output = cv.VideoWriter('panorama.mp4',cv.VideoWriter_fourcc(*'DIVX'), 20, size)
for i in range(len(panoramas)):
    output.write(panoramas[i])
output.release()
print("Panorama video generated successfully!")

OpenCV: FFMPEG: tag 0x58564944/'DIVX' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'


Panorama video generated successfully!
